In [147]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 7


In [148]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [149]:
loader = GenericLoader.from_filesystem(
    "./DotNetFramework/",
    glob="*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=10000),    
)
docs = loader.load()

In [150]:
docs

[Document(page_content='\ufeffusing System;\nusing System.Collections.Generic;\nusing System.Linq;\nusing System.Data;\nusing System.Data.SqlClient;\nusing System.Text;\nusing System.Threading.Tasks;\nusing StockManagementSystem.Models;\n\nnamespace StockManagementSystem.Repository\n{\n    public class CategoryRepository\n    {\n        string connectionString;\n        SqlConnection sqlConnection;\n        private string commandString;\n        private SqlCommand sqlCommand;\n        SqlDataAdapter sqlDataAdapter;\n        DataTable dataTable;\n\n        public CategoryRepository()\n        {\n            connectionString = @"Server=LAPTOP-BASHAROV\\SQLEXPRESS; Database=StockManagementSystemDB; Integrated Security=True";\n            sqlConnection = new SqlConnection(connectionString);\n        }\n\n        public DataTable LoadCategoryDataGridView()\n        {\n            commandString = @"SELECT * FROM Categorys";\n            sqlCommand = new SqlCommand(commandString, sqlConnectio

In [151]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=10000, chunk_overlap=0
)
documents = code_splitter.split_documents(docs)



In [152]:
documents

[Document(page_content='\ufeffusing System;\nusing System.Collections.Generic;\nusing System.Linq;\nusing System.Data;\nusing System.Data.SqlClient;\nusing System.Text;\nusing System.Threading.Tasks;\nusing StockManagementSystem.Models;\n\nnamespace StockManagementSystem.Repository\n{\n    public class CategoryRepository\n    {\n        string connectionString;\n        SqlConnection sqlConnection;\n        private string commandString;\n        private SqlCommand sqlCommand;\n        SqlDataAdapter sqlDataAdapter;\n        DataTable dataTable;\n\n        public CategoryRepository()\n        {\n            connectionString = @"Server=LAPTOP-BASHAROV\\SQLEXPRESS; Database=StockManagementSystemDB; Integrated Security=True";\n            sqlConnection = new SqlConnection(connectionString);\n        }\n\n        public DataTable LoadCategoryDataGridView()\n        {\n            commandString = @"SELECT * FROM Categorys";\n            sqlCommand = new SqlCommand(commandString, sqlConnectio

In [153]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [154]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(model="gpt-3.5-turbo")
llm=ChatOpenAI(model="gpt-4-turbo", temperature=0)


In [155]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core."),
#         ("user","old code:{oldcode}")
#     ]
# )



## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. 
Please follow below instructions to generate new source code files:
(1) New Code should be based on .NET 8+ or ASP.NET Core. 
(2) STOP Adding extra information to the source code. Any information in the generated code should be from the context
(3) DO NOT generate partial files. I need full source code for compilation.
(4) Outmost namespace is AspNetCoreApp                                          
(5) Generate the Startup.cs file 
(6) The generated source code files will be compiled immediately, STOP using HTML-encoding &lt; and &gt; for displaying purpose. Use < and > in the generated code for generic type or HTML tag
(7) Use Razor Pages page-based programming model to build web UI. Always create .cshtml and .cshtml.cs file for page implementation 
(8) For performance reason, if there are SQL query statements, continue use SQL query statement and don't convert to Entity Framework. But make sure the new code is .NET 8+ compatible
(9) If the file does not inherently use any .NET Framework-specific features that would prevent it from running under .NET 8 or later, then DO NOT generate new code file.


Please follow below instructions to output response:
(1)Please use xml tag <description></description> to contain all your description; and use xml tag <NewCode><file name="xxxx"></file><file name="yyyy"></file>...</NewCode> to wrap source code files generated.
(2)The content within <file name="xxxx"></file> is valid C# code.
                                          
In addition, basedon the context provided, please help to identify.Net Framework features replaced.
Finally, list original files which can run in .NET 8+ environment without installation of .NET framework. 

<context>
{context}
</context>
Source code files to be rewritten: {input}"""
)


In [156]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [157]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f63cc87a1d0>)

In [158]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [159]:
# response=retriever_chain.invoke({"input": documents[0].page_content})
response=retriever_chain.invoke({"input": "CustomWebControl.cs"})

In [160]:
response['answer']

'<description>The provided context includes a WebForms application using System.Web.UI, which is not supported in .NET Core or .NET 8+. Therefore, the CustomWebControl class needs to be rewritten using ASP.NET Core\'s Razor Pages model. The DBManager and repository classes use ADO.NET with SQL queries, which are compatible with .NET 8+ and do not require rewriting under the provided instructions.</description>\n\n<NewCode>\n<file name="Startup.cs">\nusing Microsoft.AspNetCore.Builder;\nusing Microsoft.AspNetCore.Hosting;\nusing Microsoft.Extensions.DependencyInjection;\nusing Microsoft.Extensions.Hosting;\n\nnamespace AspNetCoreApp\n{\n    public class Startup\n    {\n        public void ConfigureServices(IServiceCollection services)\n        {\n            services.AddRazorPages();\n        }\n\n        public void Configure(IApplicationBuilder app, IWebHostEnvironment env)\n        {\n            if (env.IsDevelopment())\n            {\n                app.UseDeveloperExceptionPage()

In [161]:
import re
from IPython.display import display, Markdown

# Your input string
input_string = response['answer']



code_match = re.search(r'<NewCode>\n(.*?)\n</NewCode>', input_string, re.DOTALL)
description_match = re.search(r'<description>\n(.*?)\n</description>', input_string, re.DOTALL)

# Extracted code and description
code = code_match.group(1) if code_match else None
description = description_match.group(1) if description_match else None

display(Markdown(description))

print(code)

<IPython.core.display.Markdown object>

<file name="Startup.cs">
using Microsoft.AspNetCore.Builder;
using Microsoft.AspNetCore.Hosting;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;

namespace AspNetCoreApp
{
    public class Startup
    {
        public void ConfigureServices(IServiceCollection services)
        {
            services.AddRazorPages();
        }

        public void Configure(IApplicationBuilder app, IWebHostEnvironment env)
        {
            if (env.IsDevelopment())
            {
                app.UseDeveloperExceptionPage();
            }

            app.UseStaticFiles();

            app.UseRouting();

            app.UseAuthorization();

            app.UseEndpoints(endpoints =>
            {
                endpoints.MapRazorPages();
            });
        }
    }
}
</file>

<file name="CustomWebControl.cshtml">
@page
@model AspNetCoreApp.Pages.CustomWebControlModel

<h1>Hello, this is a testing message.</h1>
</file>

<file name="CustomWebControl.cshtml.cs"

In [162]:
pattern = re.compile(r'<file name="(.+?)">(.*?)</file>', re.DOTALL)

# Finding all matches
matches = pattern.findall(code)

# Loop through matches and write to files
for file_name, file_content in matches:
    # Strip leading and trailing whitespaces from the content
    file_content = file_content.strip()
    # Open the file in write mode and write the content
    with open("./DotNet/"+file_name, 'w') as file:
        file.write(file_content)
    print(f"File saved: {file_name}")

File saved: Startup.cs
File saved: CustomWebControl.cshtml
File saved: CustomWebControl.cshtml.cs
